# Least Recently Used Cache
We have briefly discussed caching as part of a practice problem while studying hash maps.

The lookup operation (i.e., `get()`) and `put()` / `set()` is supposed to be fast for a cache memory.

While doing the `get()` operation, if the entry is found in the cache, it is known as a `cache hit`. If, however, the entry is not found, it is known as a `cache miss`.

When designing a cache, we also place an upper bound on the size of the cache. If the cache is full and we want to add a new entry to the cache, we use some criteria to remove an element. After removing an element, we use the `put()` operation to insert the new element. The remove operation should also be fast.

For our first problem, the goal will be to design a data structure known as a **Least Recently Used (LRU) cache**. An LRU cache is a type of cache in which we remove the least recently used entry when the cache memory reaches its limit. For the current problem, consider both `get` and `set` operations as an `use operation`.

Your job is to use an appropriate data structure(s) to implement the cache.

+ In case of a `cache hit`, your `get()` operation should return the appropriate value.
+ In case of a `cache miss`, `your get()` should return -1.
+ While putting an element in the cache, your `put()` / `set()` operation must insert the element. If the cache is full, you must write code that removes the least recently used entry first and then insert the element.

All operations must take `O(1)` time.

For the current problem, you can consider the size of `cache = 5`.


In [1]:
class DoubleNode:  
    """Doubly Linked Node object for the linked queue.
    """

    def __init__(self, value):
        self._value = value
        self._left_node = None
        self._right_node = None

    def get_left_node(self):
        """Returns left node.
        """
        return self._left_node

    def set_left_node(self, node):
        """Sets left node.
        """
        self._left_node = node
        if node: 
            node._right_node = self

    def get_right_node(self):
        """Returns right node.
        """
        return self._right_node

    def set_right_node(self, node):
        """Sets right node.
        """
        self._right_node = node
        if node:
            node._left_node = self

    def get_value(self):
        """Gets value of node.
        """
        return self._value

In [2]:
# Tests for Node Class
node = DoubleNode(10)
assert node.get_value() == 10, 'Expected value 10'
assert node.get_left_node() is None, 'Expected None left node'
node.set_left_node(DoubleNode(5))
assert node.get_left_node().get_value() == 5, 'Expected 5'
node.set_right_node(DoubleNode(20))
assert node.get_right_node().get_value() == 20, 'Expected 20'
assert node.get_right_node().get_left_node() is not None, 'Reverse link should be automatically set'
assert node.get_right_node().get_left_node().get_value() == 10, 'Value of same node expected to be 10'

print('tests completed')

tests completed


In [3]:
class Unique_Queue:
    """This class will implement a queue where each element has a unique entry.

    Whenever an element is pushed that previously exists, it's existing
    entry will be cleared and the element will be added to the tail of the queue.

    We'll implement the queue as a linked list to avoid time complexity while resizing.

    Q: head --> elem1 <--> elem2 <--> elem3 <-- tail
    """

    def __init__(self):
        self._elements = dict()  # will hold the index for each element
        self._head = None
        self._tail = None

    def push(self, value):
        """Pushes a value to the tail.
        """
        new_node = DoubleNode(value)
        if not self._head:
            # set head and tail for first element
            self._head = new_node
            self._tail = self._head
        else:
            # add new node to the right of tail element
            self._tail.set_right_node(new_node)
            self._tail = new_node

            # if element already in queue, then pop it and squeeze links
            if value in self._elements:
                # if value found at head, pop it; tail will self adjust
                if self._head.get_value() == value:
                    self.pop()
                else:
                    old_node = self._elements[value]
                    old_node.get_left_node().set_right_node(old_node.get_right_node())
            
        # update lookup
        self._elements[value] = new_node

    def pop(self):
        """Pops from the head and return the value
        """
        if self._head:
            node = self._head
            # Move head to the next node
            self._head = self._head.get_right_node()
            if self._head:
                self._head.set_left_node(None)
            
            # Remove value from lookup dict
            val = node.get_value()
            del self._elements[val]
            return node.get_value()
        
        return None



In [4]:
# tests for simple queue
q = Unique_Queue()
assert q.pop() is None, 'Q should be empty'
q.push(10)
q.push(20)
assert q.pop() == 10, 'value 10 was pushed first'
assert q.pop() == 20, 'value 20 was pushed next'
assert q.pop() is None, 'Q should be empty'

print('Tests completed')

Tests completed


In [5]:
# tests for unique queue
q = Unique_Queue()
assert q.pop() is None, 'Q should be empty'
q.push(10)
q.push(20)
assert q.pop() == 10, 'value 10 was pushed first'
assert q.pop() == 20, 'value 20 was pushed next'

q.push(10)
q.push(20)
q.push(10)
assert q.pop() == 20, '20 should be at the head and 10 went back to tail'
assert q.pop() == 10, 'value 10 is now at the head'

assert q.pop() is None, 'Q should be empty'
print('tests completed')

tests completed


In [6]:
    
class LRU_Cache(object):
    """Least Recenty Used Cache implementation.

    This is built using a Dictionary for O(1) retrieval and a
    Unique Queue for maintaining least used element.
    """

    def __init__(self, capacity):
        self._capacity = capacity
        self._contents = dict()       
        self._lru_queue =Unique_Queue()

    def get(self, key):
        # Retrieve item from provided key. Return -1 if nonexistent. 
        if key in self._contents:
            self._lru_queue.push(key)  # update usage
            return self._contents[key]
        else:
            return -1

    def set(self, key, value):
        # Set the value if the key is not present in the cache. If the cache is at capacity remove the oldest item. 
        if self._is_full():
            self._flush_lru_key()

        self._contents[key] = value
        self._lru_queue.push(key)  # update usage

    def _is_full(self):
        # Returns True if the cache is up to capacity
        if len(self._contents) >= self._capacity:
            return True 
        else:
            return False 

    def _flush_lru_key(self):
        # Removes a single entry which was the least recenty used
        oldest = self._lru_queue.pop()
        assert oldest in self._contents, f'LRU Queue has {oldest} but not the contents'
        del self._contents[oldest]



## Test Cases

In [7]:
# Basic Tests

simple_cache = LRU_Cache(2)
assert simple_cache.get(4) == -1, "Cache miss should return -1"
simple_cache.set(3, 'test')
assert simple_cache.get(3) == 'test', "Cache hit should return 'test'"
print('tests completed')

tests completed


In [8]:
our_cache = LRU_Cache(5)

our_cache.set(1, 1)
our_cache.set(2, 2)
our_cache.set(3, 3)
our_cache.set(4, 4)


In [9]:
print(our_cache.get(1))       # returns 1
print(our_cache.get(2))       # returns 2
print(our_cache.get(9))      # returns -1 because 9 is not present in the cache


1
2
-1


In [10]:
our_cache.set(5, 5) 
our_cache.set(6, 6)

our_cache.get(3)      # returns -1 because the cache reached it's capacity and 3 was the least recently used entry

-1